In [4]:
# Avoids conflicts with Client for dask
import pystac_client
import planetary_computer
import leafmap
import stackstac
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:37515/status,
Dashboard: http://127.0.0.1:37515/status,Workers: 4
Total threads: 16,Total memory: 7.64 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35081,Workers: 4
Dashboard: http://127.0.0.1:37515/status,Total threads: 16
Started: Just now,Total memory: 7.64 GiB
Comm: tcp://127.0.0.1:33735,Total threads: 4
Dashboard: http://127.0.0.1:34637/status,Memory: 1.91 GiB
Nanny: tcp://127.0.0.1:41053,


In [5]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [6]:
bbox = (32.16033157094111, 22.852984440390316, 33.288140399555346, 23.806193394664234)

In [7]:
search = catalog.search(
    collections=['landsat-8-c2-l2'],
    bbox=bbox,
)

In [8]:
items = search.get_all_items()
len(items)

804

In [16]:
type(items)

pystac.item_collection.ItemCollection

In [9]:
%%time
stack = stackstac.stack(items, bounds_latlon=bbox)
stack

CPU times: user 2.39 s, sys: 51.4 ms, total: 2.44 s
Wall time: 2.38 s


<xarray.DataArray 'stackstac-cb220b80e0b83f3f62e762a772c9543c' (time: 804,
                                                                band: 19,
                                                                y: 3526, x: 3858)> Size: 2TB
dask.array<fetch_raster_window, shape=(804, 19, 3526, 3858), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/27)
  * time                         (time) datetime64[ns] 6kB 2013-04-03T08:19:3...
    id                           (time) <U31 100kB 'LC08_L2SP_175043_20130403...
  * band                         (band) <U13 988B 'SR_B1' ... 'SR_QA_AEROSOL'
  * x                            (x) float64 31kB 4.138e+05 ... 5.296e+05
  * y                            (y) float64 28kB 2.633e+06 ... 2.527e+06
    view:off_nadir               int64 8B 0
    ...                           ...
    gsd                          (band) float64 152B 30.0 30.0 ... 30.0 30.0
    title                        (band) <U46 3kB 'Coastal/Aerosol Band (B1)' ...
    common_name                  (band) object 152B 'coastal' 'blue' ... None
    center_wavelength            (band) object 152B 0.44 0.48 0.56 ... None None
    full_width_half_max          (band) object 152B 0.02 0.06 0.06 ... None None
    epsg                         int64 8B 32636
Attributes:
    spec:        RasterSpec(epsg=32636, bounds=(413850.0, 2527260.0, 529590.0...
    crs:         epsg:32636
    transform:   | 30.00, 0.00, 413850.00|\n| 0.00,-30.00, 2633040.00|\n| 0.0...
    resolution:  30.0

In [10]:
stack = stack.assign_coords(band=stack.common_name.fillna(stack.band).rename("band"))
stack.band

<xarray.DataArray 'band' (band: 19)> Size: 152B
array(['coastal', 'blue', 'green', 'red', 'nir08', 'swir16', 'swir22', 'ST_QA',
       'lwir11', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_TRAD', 'ST_URAD',
       'QA_PIXEL', 'ST_ATRAN', 'ST_CDIST', 'QA_RADSAT', 'SR_QA_AEROSOL'],
      dtype=object)
Coordinates: (12/15)
    view:off_nadir             int64 8B 0
    proj:epsg                  int64 8B 32636
    platform                   <U9 36B 'landsat-8'
    instruments                object 8B {'oli', 'tirs'}
    landsat:collection_number  <U2 8B '02'
    landsat:wrs_type           <U1 4B '2'
    ...                         ...
    title                      (band) <U46 3kB 'Coastal/Aerosol Band (B1)' .....
    common_name                (band) object 152B 'coastal' 'blue' ... None None
    center_wavelength          (band) object 152B 0.44 0.48 0.56 ... None None
    full_width_half_max        (band) object 152B 0.02 0.06 0.06 ... None None
    epsg                       int64 8B 32636
  * band                       (band) object 152B 'coastal' ... 'SR_QA_AEROSOL'

In [11]:
stack.sel(band=["red", "green", "blue"])

<xarray.DataArray 'stackstac-cb220b80e0b83f3f62e762a772c9543c' (time: 804,
                                                                band: 3,
                                                                y: 3526, x: 3858)> Size: 262GB
dask.array<getitem, shape=(804, 3, 3526, 3858), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/27)
  * time                         (time) datetime64[ns] 6kB 2013-04-03T08:19:3...
    id                           (time) <U31 100kB 'LC08_L2SP_175043_20130403...
  * x                            (x) float64 31kB 4.138e+05 ... 5.296e+05
  * y                            (y) float64 28kB 2.633e+06 ... 2.527e+06
    view:off_nadir               int64 8B 0
    view:sun_elevation           (time) float64 6kB 60.36 61.0 ... 56.4 57.14
    ...                           ...
    title                        (band) <U46 552B 'Red Band (B4)' ... 'Blue B...
    common_name                  (band) object 24B 'red' 'green' 'blue'
    center_wavelength            (band) object 24B 0.65 0.56 0.48
    full_width_half_max          (band) object 24B 0.04 0.06 0.06
    epsg                         int64 8B 32636
  * band                         (band) object 24B 'red' 'green' 'blue'
Attributes:
    spec:        RasterSpec(epsg=32636, bounds=(413850.0, 2527260.0, 529590.0...
    crs:         epsg:32636
    transform:   | 30.00, 0.00, 413850.00|\n| 0.00,-30.00, 2633040.00|\n| 0.0...
    resolution:  30.0